# Recapitulate compressed, spooled data into a series of tiles

This notebook is used together with PYME.experimental.watcher to spool and compress files coming from 3rd party acquisition software into PYMEs compressed format. This notebook handles reading the compressed data and re-exporting as a series of tiles.

## Pre-requisites

### Environment

- python
- python-microscopy
- pyme-compress

Easiest way to get this (at present with broken conda packages) is as follows: 

- Starting from a miniforge install with mamba and with conda-forge in the channels list, create a new environment:
`mamba create -c david_baddeley -n PYME python=3.X pyme-depends`  where x in is between 8 and 11 inclusive
- make sure you have the MS Visual C version matiching your python version (should be a free download, if you don't know which version, you can always wait until you get an error from python setup.py develop).
- activate the environment: `conda activate PYME`
- change to a suitable directory and checkout the latest PYME from the github repository:
`git clone https://github.com/python-microscopy/python-microscopy.git`
- `cd python-microscopy`
- `python setup.py develop`

Optionally test by trying to run the viewer `PYMEImage --test3D` (this could fail on wxpython issues even if you have an install which will work as a library)

### RAM drive

- create a RAM drive to spool to (optional in first instance, but desirable for speed eventually)

### PYMEDataDir

If the compressed data should be saved somewhere other than your home directory, the PYMEDataDir config option needs to be set.

- Open (or if necescesary create) `config.yaml` in `{home}/.PYME`
- add a line `dataserver-root: "/path/to/directory/for/compressed.data"

### Data acquisition

- `conda activate PYME`
- `PYMEClusterOfOne` (this launches the server which will recieve and save compressed frames - this could also be a full multicomputer PYME cluster, but ClusterOfOne is a good place to start)
- Start spooling in your acquisition software (.tif to a directory of your choosing, ideally on ram disk)
- `python -m PYME.experimental.watcher /path/to/acquisition/tif/directory PYMECLUSTER:///path/to/somefile.pcs`  ie `python -m PYME.experimental.watcher input_directory output_series`
- Wait for acquisition to finish
- hit `Ctrl-C` in the terminal running `PYME.experimental.watcher`

**FIXME: The watcher currently uses hard-coded values for camera ADOffset and electrons per count. I think these are suitable for the PYME simulator, NOT for your camera. This needs to be altered and/or exposed as config items** 

### Reconstruction (this notebook)

- This should be run in your `PYME` environment created earlier.
- The data server (PYMEClusterOfOne) must be running

In [14]:
import os
import numpy as np
from PYME.IO.image import ImageStack
from PYME.IO.DataSources.BaseDataSource import XYZTCWrapper

In [10]:
# Load the data set
# this should show up as a large time series
# NB - This is a lazy load, nothing is pulled into memory at this point
# FIXME - adjust for path to your spooled data - This should be PYME-CLUSTER:///path/relative/to ~/PYMEData
im = ImageStack(filename='PYME-CLUSTER:///PYMECLUSTER/david/2025_3_17/test1.pcs')

filename == PYME-CLUSTER:///PYMECLUSTER/david/2025_3_17/test1.pcs


/Users/david/python-microscopy/PYME/Analysis/MetaData.py:251: UserWarning: ADOffset fudged as 967 and probably wrong

To change ADOffset, execute the following in the console: image.mdh['Camera.ADOffset'] = newValue

Or use the Metadata pane in the GUI (right click on value to change)
  warnings.warn("ADOffset fudged as %d and probably wrong\n\nTo change ADOffset, execute the following in the console: image.mdh['Camera.ADOffset'] = newValue\n\nOr use the Metadata pane in the GUI (right click on value to change)" % md.getEntry('Camera.ADOffset'))#,'Did not find laser turn on signature')

To change ADOffset, execute the following in the console: image.mdh['Camera.ADOffset'] = newValue

Or use the Metadata pane in the GUI (right click on value to change)


In [12]:
# what shape is the data (this should be (x, y,1,n_frames, 1))
print('Current shape:', im.data_xyztc.shape)

Current shape: (256, 256, 1, 23280, 1)


In [16]:
# re-dimension the data so that it has the correct dimensions (i.e. (x, y, n_z_slices, n_tiles, n_colours)). We'll use the time dimension for our tiles

#Parameters - modify as appropriate
#####################################

#dimorder - order in which dimensions were acquired (from fastest to slowest)
#modify this as appropriate if acquisition order was different - e.g. XYCZT if colours were switched on every frame, followed by z-steping, followed by tiles
dimorder = 'XYZTC' 

#ditto for the below
# TODO - read from a metadata file dropped with the image data
n_z_slices=10
n_colours=2

# compute number of tiles
n_tiles = im.data_xyztc.getNumSlices()/(n_z_slices*n_colours)
print('n_tiles computed as:', n_tiles)

# sanity check - n_tiles should be an integer if total number of slices is evenly divisible by z_slices*colours
assert((n_tiles%1) == 0)
# if above assertion passed, safe to cast to int
n_tiles= int(n_tiles)

#actually do the redimensioning
#again, this is lazy and the frames are not in memory - the wrapper just changes how they are indexed
d = XYZTCWrapper(im.data_xyztc)
d.set_dim_order_and_size(dimorder, size_z=n_z_slices, size_t=n_tiles, size_c=n_colours)


n_tiles computed as: 1164.0


In [25]:
# save each tile as a multichannel .tif z-stack
# modify as appropriate
outputdir = '/Users/david/test_chunck_extract'

for i in range(n_tiles):
    # using PYME .tif save
    # by default, decompressed data uses a float32 data type
    #ImageStack(d[:,:,:,i,:]).save(os.path.join(outputdir, f'tile_float{i:04d}.tif'))
    
    # for most purposes, we probably want it as uint16 (i.e. the same as the original data)
    ImageStack(d[:,:,:,i,:].astype('uint16')).save(os.path.join(outputdir, f'tile{i:04d}.tif'))
    
    
    #alternatively, save each channel to a separate stack
    # uncomment if preffered
    #for c in range(n_colours):
    #    ImageStack(d[:,:,:,i,c].astype('uint16')).save(os.path.join(outputdir, f'tile{i:04d}_chan{c:02d}.tif'))
    
